In [ ]:
%cd ../mmk2

In [ ]:
import os
import sys

sys.path = [p for p in sys.path if "Auto" not in p]

from functools import partial
from mmk.data import make_root_db, AudioFileWalker, file_to_fft, Database

from mmk.kit.neptune_project import upload_database

In [ ]:
# make a walker to combine (different) sources of audio_files

walker = AudioFileWalker(
    roots=["../gould", ],
    files=[]
)

# to use multithreading, we need to pass a function 
# that returns an array and a dict of attributes (hop, n_fft...)
# `file_to_fft` does that, `partial` allow us to specify its params without evaluating it

extract = partial(file_to_fft, n_fft=256, hop_length=256//4, sr=22050)

# pass the walker and the extact function to `make_root_db` to build an on-disk dataset

make_root_db("../gould.h5", walker, extract_func=extract)

In [ ]:
# check that everything's good
db = Database("../gould.h5")

# we stored the params!
db.fft.attrs

In [ ]:
api_token = os.environ.get("NEPTUNE_API_TOKEN")

upload_database(db, api_token, "k-tonal/data", "Arie-Nessun-Dorma")

In [ ]:
# we can also append data to the db (!!!! THIS IS IRREVERSIBLE !!! (for now...))

db.fft.add(db.fft[:1000], filename="some more stuff")

# we have to reload the db for it to take effect
db = Database("../gould.h5")

# chekcout the last on the list
db.metadata.iloc[-1:]